In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import glob
import wandb
import os
import torch.optim as optimizers
import pandas as pd
from copy import deepcopy
from ml_collections import ConfigDict 
import tqdm
import torch.nn.functional as F
import torch
import torch.nn as nn
from collections import defaultdict
import yaml
from sklearn.metrics import roc_auc_score
import networkx as nx

import sys
sys.path.append("../../src")
import dfs_code 
from dfs_transformer import collate_downstream, DFSCodeSeq2SeqFC, TrainerGNN
torch.multiprocessing.set_sharing_strategy('file_system')

import torch_geometric.nn as tnn
import networkx as nx

2022-01-31 13:05:53.267892: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt/lib:/opt/intel/tbb/lib:/opt/intel/clck/2019.0/lib:/opt/intel/compilers_and_libraries_2019/linux/lib:/opt/intel/compilers_and_libraries/linux/lib:/opt/intel/itac/2019.0.018/lib:/opt/intel/itac_2019/intel64/lib:/opt/intel/itac_latest/intel64/lib:/opt/intel/parallel_studio_xe_2019.0.045/clck_2019/lib:/opt/intel/parallel_studio_xe_2019.0.045/itac_2019/l

In [3]:
config = wandb.config
config.graph_file = "/mnt/ssd/datasets/graphs/reddit_threads/reddit_edges.json"
config.label_file = "/mnt/ssd/datasets/graphs/reddit_threads/reddit_target.csv"
config.batch_size = 50
config.n_epochs = 100
config.learning_rate = 0.01
config.rep = 1
config.max_edges = 50
config.n_samples = 10000
config.n_channels = 32
config.n_layers = 2
config.model = "tnn.models.GCN"
config.readout = "tnn.global_mean_pool"
config.training = {}

In [4]:
run = wandb.init(mode="online", project="karateclub-reddit-50", entity="dfstransformer", 
                 config=config, job_type="evaluation")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chrisxx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-01-31 13:05:56.283491: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt

In [5]:
with open(config.graph_file, 'r') as f:
    graph_dict = json.load(f)
label_df = pd.read_csv(config.label_file)

In [6]:
def graph2labelledgraph(graph, use_dummy=False):
    graph = deepcopy(graph)
    node_ids = np.unique(graph).tolist()
    dummy = max(node_ids) + 1
    
    edge_labels = len(graph)*[0]
    edgeindex = []
    for e in graph:
        edgeindex += [e]
        edgeindex += [[e[1], e[0]]]
        edge_labels += [0]
    
    if use_dummy:
        for idx in node_ids:
            edgeindex += [[idx, dummy], [dummy, idx]]
            edge_labels += [1, 1]
        node_ids += [dummy]
    
    edgeindex = np.asarray(edgeindex).T
    
    node_labels = []
    for idx in node_ids:
        node_labels += [(edgeindex[0] == idx).sum()]
    return edgeindex, node_labels, edge_labels

In [7]:
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader

class KarateClubDataset(Dataset):
    def __init__(self, graph_file, label_file, max_n = None, max_edges=50):
        super().__init__()
        self.graph_file = graph_file
        self.label_file = label_file
        with open(graph_file, 'r') as f:
            self.graph_dict = json.load(f)
        self.label_df = pd.read_csv(label_file)
        self.data = []
        self.maxn = max_n
        self.max_edges = max_edges
        self.preprocess()
    
    def preprocess(self):
        maxn = self.maxn
        maxdegree = 0
        edgeindex_list = []
        vlabels_list = []
        elabels_list = []
        label_list = []
        for idx, (graph, label) in tqdm.tqdm(enumerate(zip(self.graph_dict.values(), self.label_df['target']))):  
            edgeindex, vlabels, elabels = graph2labelledgraph(graph)
            if len(elabels)//2 > self.max_edges:
                continue
            maxdegree = max(maxdegree, max(vlabels))
            edgeindex_list += [edgeindex]
            vlabels_list += [vlabels]
            elabels_list += [elabels]
            label_list += [self.label_df['target'][idx]]
            if maxn is not None:
                if idx >= maxn:
                    break
        self.maxdegree = maxdegree
        
        for idx, (edgeindex, vlabels, elabels, label) in tqdm.tqdm(enumerate(zip(edgeindex_list,
                                                                                vlabels_list,
                                                                                elabels_list, 
                                                                                label_list))):
            g = nx.Graph()
            g.add_edges_from(edgeindex.T.tolist())
            clustering_coefs = nx.clustering(g)
            clustering_coefs = torch.tensor([clustering_coefs[idx] for idx in range(len(clustering_coefs))])
            eccentricity = nx.eccentricity(g)
            eccentricity = torch.tensor([eccentricity[idx] for idx in range(len(eccentricity))], dtype=torch.long)
            
            #centrality_ev = nx.eigenvector_centrality(g, max_iter=1000, tol=1e-4)
            #centrality_deg = nx.degree_centrality(g)
            #centrality_closeness = nx.closeness_centrality(g)
            #centrality_sp = nx.betweenness_centrality(g)
            #centrality_subgraph = nx.subgraph_centrality(g)
            #centrality = []
            #for idx in range(len(centrality_ev)):
            #    centrality += [[centrality_ev[idx],
            #                    centrality_deg[idx],
            #                    centrality_closeness[idx],
            #                    centrality_sp[idx],
            #                    centrality_subgraph[idx]]]
            deg_feats = F.one_hot(torch.tensor(vlabels), num_classes=maxdegree+1).float()
            ecc_feats = F.one_hot(eccentricity, num_classes=self.max_edges).float()
            clust_feats = clustering_coefs.unsqueeze(1)
            node_features = torch.cat((deg_feats, ecc_feats, clust_feats), dim=1)
            edge_features = F.one_hot(torch.tensor(elabels), num_classes=2).float()
            self.data += [Data(**{"edge_index": torch.tensor(edgeindex, dtype=torch.long),
                                "x": node_features, #torch.tensor(centrality),
                                "centrality_deg": node_features,
                                "y": torch.tensor(label, dtype=torch.long),
                                "num_nodes": len(node_features)})]
                                #"centrality_ev": torch.tensor([centrality_ev[idx] for idx in range(len(centrality_ev))])})]

    def len(self):
        return len(self.data)

    def get(self, idx):
        return self.data[idx]
        

In [8]:
dataset = KarateClubDataset(config.graph_file, config.label_file, max_n=config.n_samples, max_edges=config.max_edges)

10000it [00:01, 5152.70it/s]
9042it [00:05, 1643.34it/s]


In [9]:
dim_input = dataset[0].x.shape[1]

In [10]:
from einops import rearrange
model = tnn.Sequential('x, edge_index, batch', [
    (eval(config.model)(dim_input, config.n_channels, config.n_layers), 'x, edge_index -> x'),
    (eval(config.readout), 'x, batch -> x'),
    (nn.Linear(config.n_channels, 2),'x -> x')
])

In [11]:
n = len(dataset)
n_train = int(0.8*n)
n_valid = 0
n_test = n - n_train - n_valid

perm = np.random.permutation(len(dataset))
train_idx = torch.tensor(perm[:n_train], dtype=torch.long)
valid_idx = torch.tensor(perm[n_train:n_train+n_valid].tolist(), dtype=torch.long)
test_idx = torch.tensor(perm[n_train+n_valid:].tolist(), dtype=torch.long)

In [12]:
ce = nn.CrossEntropyLoss() 

In [13]:
def collate_fn(data):
    return data, data.y

In [14]:
trainloader = DataLoader(dataset, sampler=torch.utils.data.SubsetRandomSampler(train_idx), 
                         batch_size=config.batch_size, collate_fn=collate_fn, num_workers=8)
validloader = DataLoader(dataset, sampler=torch.utils.data.SubsetRandomSampler(valid_idx), 
                         batch_size=config.batch_size, num_workers=8)
testloader = DataLoader(dataset, sampler=torch.utils.data.SubsetRandomSampler(test_idx), 
                        batch_size=config.batch_size, num_workers=8)

In [15]:
def loss(pred, y, ce=ce):
    return ce(pred, y.squeeze())

def auc(pred, y):
    return roc_auc_score(y.detach().cpu().numpy(), np.argmax(pred.detach().cpu().numpy(), axis=1))
    #return roc_auc_score(y.detach().cpu().numpy(), pred[:, 1].detach().cpu().numpy())
    

In [16]:
trainer = TrainerGNN(model, trainloader, loss, metrics={'auc': auc}, n_epochs=config.n_epochs, lr=config.learning_rate, validloader=testloader, 
                  es_period=n_train//config.batch_size, lr_adjustment_period=10*n_train//config.batch_size//4, wandb_run=run,
                  clip_gradient_norm=None, lr_patience=5, es_patience=20)

In [17]:
data = next(iter(trainloader))

In [18]:
data.x

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0278],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.1667],
        [0.0000, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [19]:
trainer.fit()

Valid 1: loss 0.521077 0.8039:  68%|████████████████████████████████████████████████████████████████████████████████████████████████▌                                              | 25/37 [00:00<00:00, 39.24it/s]


Valid 2: loss 0.516337 0.7776:  65%|████████████████████████████████████████████████████████████████████████████████████████████▊                                                  | 24/37 [00:00<00:00, 37.17it/s]


Valid 3: loss 0.501358 0.7949:  32%|██████████████████████████████████████████████▍                                                                                                | 12/37 [00:00<00:01, 24.05it/s]


Epoch 3: loss 0.518153 0.7454: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 145/145 [00:03<00:00, 37.34it/s]


EarlyStopping counter: 1 out of 20


...

Epoch 21: loss 0.497906 0.7649:  96%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏     | 139/145 [00:03<00:00, 22.58it/s]

EarlyStopping counter: 19 out of 20


Valid 22: loss 0.531871 0.6823:  54%|████████████████████████████████████████████████████████████████████████████▊                                                                 | 20/37 [00:00<00:00, 38.02it/s]


Epoch 22: loss 0.495530 0.7607:  84%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                      | 122/145 [00:03<00:00, 36.22it/s]

EarlyStopping counter: 20 out of 20
